<a href="https://colab.research.google.com/github/viswavich/DABOTICS-TASK-3-PYTHON-INTERN/blob/main/URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install flask pyngrok

# Set your ngrok authtoken
from pyngrok import ngrok

ngrok.set_auth_token('2jMqkBE1YRakjFDUos5ji7Blg39_7acurSLwisxB57NrwL3W1')

In [31]:
from flask import Flask, request, redirect, render_template_string
import sqlite3
import string
import random

app = Flask(__name__)

# Connect to the SQLite database
def get_db_connection():
    conn = sqlite3.connect('database.db')
    conn.row_factory = sqlite3.Row
    return conn

# Create table
def init_db():
    conn = get_db_connection()
    conn.execute('CREATE TABLE IF NOT EXISTS urls (id INTEGER PRIMARY KEY, original_url TEXT, short_url TEXT)')
    conn.commit()
    conn.close()

# Generate a random short URL
def generate_short_url(length=6):
    characters = string.ascii_letters + string.digits
    short_url = ''.join(random.choice(characters) for _ in range(length))
    return short_url

# Route to shorten URL
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        original_url = request.form['original_url']
        short_url = generate_short_url()

        conn = get_db_connection()
        conn.execute('INSERT INTO urls (original_url, short_url) VALUES (?, ?)', (original_url, short_url))
        conn.commit()
        conn.close()

        return render_template_string(index_template, short_url=short_url)

    return render_template_string(index_template)

# Redirect to the original URL
@app.route('/<short_url>')
def redirect_to_url(short_url):
    conn = get_db_connection()
    url_data = conn.execute('SELECT original_url FROM urls WHERE short_url = ?', (short_url,)).fetchone()
    conn.close()

    if url_data:
        return redirect(url_data['original_url'])
    else:
        return 'URL not found', 404

index_template = """
<!doctype html>
<html>
<head>
    <title>URL Shortener</title>
</head>
<body>
    <h1>URL Shortener</h1>
    <form method="post">
        <label for="original_url">Enter URL to shorten:</label>
        <input type="text" id="original_url" name="original_url" required>
        <button type="submit">Shorten</button>
    </form>

    {% if short_url %}
        <p>Shortened URL: <a href="/{{ short_url }}">{{ short_url }}</a></p>
    {% endif %}
</body>
</html>
"""

init_db()

In [35]:
from pyngrok import ngrok

# Terminate any existing tunnels
ngrok.kill()

# Start a new ngrok tunnel
public_url = ngrok.connect(5000)
print(f'Public URL: {public_url}')

Public URL: NgrokTunnel: "https://d4ac-35-194-221-38.ngrok-free.app" -> "http://localhost:5000"


In [34]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
